In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import os
import sys
import dask
import pandas as pd
import xarray as xr
import dask.array as da
import numpy as np
from re import split
from scipy.interpolate import griddata
from scipy.spatial import cKDTree as KDTree
import random
from skimage.filters import gaussian, threshold_otsu
from skimage import measure
from dask import delayed
from dask_image.ndfilters import uniform_filter as uf
from dask_image.ndmeasure import variance as varian
import dask.dataframe as dd
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, progress
from datetime import datetime, timezone

sys.path.insert(1, f"{os.path.abspath(os.path.join(os.path.abspath(''), '../'))}")
from src.utils import get_pars_from_ini
# from src.ufucnt_xr import lee_filter_new_

location = split(', |_|-|!', os.popen('hostname').read())[0].replace("\n", "")
path_data = get_pars_from_ini(campaign='loc')[location]['path_data']
path_proj = get_pars_from_ini(campaign='loc')[location]['path_proj']



In [2]:
# @jit
def get_col_row(x, size=30):
    ncols = da.ptp(x) / size
    return int(ncols)


def excluding_mesh(x, y, nx=30, ny=30):
    """
    Construct a grid of points, that are some distance away from points (x,
    """

    dx = x.ptp() / nx
    dy = y.ptp() / ny

    xp, yp = np.mgrid[x.min() - 2 * dx:x.max() + 2 * dx:(nx + 2) * 1j,
             y.min() - 2 * dy:y.max() + 2 * dy:(ny + 2) * 1j]
    xp = xp.ravel()
    yp = yp.ravel()

    tree = KDTree(np.c_[x, y])
    dist, j = tree.query(np.c_[xp, yp], k=1)

    # Select points sufficiently far away
    m = (dist > np.hypot(dx, dy))
    return xp[m], yp[m]


def regridd(data, x, y, size=30):
    """
    data = xarray datarray
    size = desired pixel size in meters
    """
    if data.ndim > 2:
        x = da.moveaxis(x.reshape(-1, x.shape[-1]), 0, -1)
        y = da.moveaxis(y.reshape(-1, y.shape[-1]), 0, -1)
        ncols_n = max(np.apply_along_axis(get_col_row, arr=x, axis=1))
        nrows_n = max(np.apply_along_axis(get_col_row, arr=y, axis=1))
        x_new_n = da.from_array(np.moveaxis(np.linspace(np.amin(x, -1), np.amax(x, -1), ncols_n), 0, -1))
        y_new_n = da.from_array(np.moveaxis(np.linspace(np.amax(y, -1), np.amin(y, -1), nrows_n), 0, -1))
        mesh = [delayed(da.meshgrid)(x_new_n[i], y_new_n[i]) for i in range(x_new_n.shape[0])]

        z_n = da.rollaxis(data.reshape(-1, data.shape[-1]), 1)
        idx_n = x.argsort(axis=-1)
        x = np.take_along_axis(x, idx_n, axis=-1)
        y = np.take_along_axis(y, idx_n, axis=-1)
        z_n = np.take_along_axis(z_n, idx_n, axis=-1)

        vp_n = dask.compute(*[delayed(excluding_mesh)(x[i], y[i]) for i in range(x.shape[0])])
        xn = [vp_n[i][0] for i in range(len(vp_n))]
        yn = [vp_n[i][1] for i in range(len(vp_n))]
        zn = dask.compute(*[delayed(da.zeros_like)(xn[i]) for i in range(x.shape[0])])
        xi_ = [mesh[i][0] for i in range(len(vp_n))]
        xi_ = dask.compute(*[da.from_delayed(v, shape=(x.shape[0], np.nan), dtype=float) for v in xi_])
        yi_ = [mesh[i][1] for i in range(len(vp_n))]
        yi_ = dask.compute(*[da.from_delayed(v, shape=(x.shape[0], np.nan), dtype=float) for v in yi_])
        zr = [delayed(griddata)((np.r_[x[i, :], xn[i]], np.r_[y[i, :], yn[i]]), np.r_[z_n[i, :], zn[i]],
                                (xi_[i], yi_[i]), method='linear', fill_value=0)
              for i in range(x.shape[0])]
        zr = da.dstack(dask.compute(*zr))
        xi_ = da.rollaxis(da.rollaxis(da.asarray(xi_), axis=-1), axis=-1)
        yi_ = da.rollaxis(da.rollaxis(da.asarray(yi_), axis=-1), axis=-1)
        return zr, xi_, yi_

    else:
        x_s = x.flatten()
        y_s = y.flatten()
        data = data.compute().flatten()
        idx = x_s.argsort()
        x_s, y_s = np.take_along_axis(x_s, idx, axis=0), np.take_along_axis(y_s, idx, axis=0)
        data = np.take_along_axis(data, idx, axis=0)
        ncols = get_col_row(x=x_s, size=size)
        nrows = get_col_row(x=y_s, size=size)
        x_new = np.linspace(x_s.min(), x_s.max(), int(ncols))
        y_new = np.linspace(y_s.max(), y_s.min(), int(nrows))
        xi, yi = np.meshgrid(x_new, y_new)
        xp, yp = excluding_mesh(x_s, y_s, nx=35, ny=35)
        zp = np.nan + np.zeros_like(xp)
        z0 = griddata((np.r_[x_s, xp], np.r_[y_s, yp]), np.r_[data, zp], (xi, yi), method='linear', 
                      fill_value=-9999)
        return z0, xi, yi


def lee_filter_new(img, size, tresh=-150):
    if img.ndim == 2:
        shape = (size, size)
    else:
        shape = (size, size, 1)
    img = da.where(da.logical_or(da.isnan(img), da.equal(img, -9999)), tresh, img)
    img_mean = uf(img, shape)
    img_sqr_mean = uf(da.power(img, 2), shape)
    img_variance = img_sqr_mean - da.power(img_mean, 2)
    overall_variance = varian(img)
    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    img_output = da.where(img_output > 0, img_output, 0)
    return img_output


def process_new(zhh14, x, y, time):
    x = x[:, 0, :, :]
    img_filtered = lee_filter_new(zhh14, size=3, tresh=-200)
    img, xi, yi = regridd(img_filtered, x, y)
    px_tot, _, _ = regridd(np.ones_like(zhh14), x, y)
    px_tot = np.where(px_tot < 0.99, 0, px_tot)
    px_tot = np.apply_along_axis(np.count_nonzero, arr=px_tot, axis=1)
    num_px = np.apply_along_axis(np.count_nonzero, arr=img, axis=1)
    img = np.where(img > 0., img, 0.)
    blurred = gaussian(img, sigma=0.8)
    binary = blurred > threshold_otsu(blurred)
    labels = measure.label(binary)
    if (labels.ndim > 2) & (zhh14.shape[-1] != 1):
        max_zhh14 = np.apply_along_axis(np.max, arr=img, axis=0).compute()
        max_zhh14 = np.apply_along_axis(np.max, arr=max_zhh14, axis=0)
        df_max_zhh = pd.Series(max_zhh14, index=time, name='max_zhh')
        df_num_px = pd.DataFrame(np.moveaxis(num_px.compute(), 0, -1), index=time)
        df_num_px.columns = pd.MultiIndex.from_product([['num_px'], df_num_px.columns])
        df_tot = pd.DataFrame(np.moveaxis(px_tot.compute(), 0, -1), index=time)
        df_tot.columns = pd.MultiIndex.from_product([['tot_px'], df_tot.columns])
        tab_props = [measure.regionprops_table(labels[:, :, i], img[:, :, i],
                                               properties=['area', 'perimeter', 'bbox', 'major_axis_length',
                                                           'minor_axis_length']) for i in range(labels.shape[-1])]

        df = pd.DataFrame(data=tab_props, index=pd.to_datetime(time))
        df = df.merge(df_max_zhh, left_index=True, right_index=True)
        df = df.merge(df_num_px, left_index=True, right_index=True)
        df = df.merge(df_tot, left_index=True, right_index=True)
    else:
        max_zhh = np.max(img.compute())
        props = measure.regionprops_table(labels[:, :, 0], img[:, :, 0].compute(),
                                          properties=['area', 'perimeter', 'bbox', 'major_axis_length',
                                                      'minor_axis_length'])
        props['max_zhh'] = max_zhh
        df = pd.DataFrame(data=props, index=pd.to_datetime(time))

    dates = datetime.now(timezone.utc)
    df.to_csv(f"../results/all_filtered_{dates:%Y}{dates:%m}{dates:%d}{dates:%H}{dates:%M}_Wn_zhh35.csv")
    return 'done!'


def ufunc_wrapper(data, zhh='zhh14'):
    x = data.range * data.DR * np.sin(np.deg2rad(data.azimuth))  # add roll
    y = data.alt3d
    zhh = data[zhh].where(data.alt3d > 500)
    _data = [zhh, x, y, data.time]
    icd = [list(i.dims) for i in _data]
    dfk = {'allow_rechunk': True, 'output_sizes': {}}
    a = xr.apply_ufunc(process_new,
                   *_data,
                   input_core_dims=icd,
                   output_core_dims=[[], ],
                   dask_gufunc_kwargs=dfk,
                   dask='parallelized',
                   vectorize=True,
                   output_dtypes=[(str)]
                   )
    return a

In [3]:
# client.close()
# cluster.close()

In [4]:
cluster = SLURMCluster(queue="seseml",
                       memory='200GB',
                       cores=40,
                       processes=1,
                       walltime='48:00:00',
                       scheduler_options={'host': '172.22.179.3:7225', 'dashboard_address': ':7799'})

In [5]:
# cluster.scale(2)
cluster.adapt(maximum_jobs=8)
cluster

In [6]:
%%bash
squeue -u alfonso8

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [7]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://172.22.179.3:7799/status,
Dashboard: http://172.22.179.3:7799/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://172.22.179.3:7225,Workers: 0
Dashboard: http://172.22.179.3:7799/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [8]:
%%time
ds_xr = xr.open_zarr(f'{path_data}/zarr_rckd/KUsKAsWs/lores.zarr')
# ds_xr = xr.open_zarr(f'{path_data}/zarr/KUsKAs_Wn/lores.zarr')
ds_xr = ds_xr.sel(time=~ds_xr.get_index("time").duplicated())



CPU times: user 262 ms, sys: 22.3 ms, total: 284 ms
Wall time: 280 ms


In [9]:
print(len(ds_xr.time))

13969


In [10]:
%%time
ds_data = ds_xr[['zhh35', 'azimuth', 'DR']]

len(ds_data.time)

CPU times: user 113 µs, sys: 61 µs, total: 174 µs
Wall time: 178 µs


13969

In [11]:
a = ufunc_wrapper(ds_data,zhh='zhh35')

In [12]:
%%time
w = dask.compute(a)

distributed.scheduler - ERROR - Couldn't gather keys {"('zeros_like-ed95666ad83dbdbbc37fc58ac1730fde', 0)": []} state: ['processing'] workers: []
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: [], ('zeros_like-ed95666ad83dbdbbc37fc58ac1730fde', 0)
NoneType: None
distributed.scheduler - ERROR - Couldn't gather keys {"('zeros_like-ed95666ad83dbdbbc37fc58ac1730fde', 0)": []} state: [None] workers: []
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: [], ('zeros_like-ed95666ad83dbdbbc37fc58ac1730fde', 0)
NoneType: None
distributed.scheduler - ERROR - Couldn't gather keys {"('zeros_like-ed95666ad83dbdbbc37fc58ac1730fde', 0)": []} state: [None] workers: []
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: [], ('zeros_like-ed95666ad83dbdbbc37fc58ac1730fde', 0)
NoneType: None
distributed.scheduler - ERROR - Couldn't gather keys {"('zeros_like-174cf9e2601ced03c796ea4cf31edd6b', 0)": []} state: ['process

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-1f177776-b975-4af8-b860-2a4149487294
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-d907c46e-313d-4465-939f-06ebb1b20977
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-89e6c93e-1bfe-4a0b-8890-0d46875e06c9
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-df0c7397-e4a7-4fe7-aff6-60b64a2dbd86
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-65e82120-0756-466e-af99-5763b6c44298
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-d2cd872d-5101-42f6-bbe5-c637a31ba1d2
NoneType: None
distributed.scheduler - ERROR - Workers don't have p

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-08699d94-0871-4cf7-a2bd-b4dd7c00c66a
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-59ec0c1f-e968-4c5e-bc75-cb0ebff1b55f
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-bd173271-7017-42df-9e3c-d7582f6d62b4
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-342de2b0-f35f-46cc-acec-20892964c005
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-819c4adc-ad78-49b5-bbdc-11b62cd7d24d
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-425d523b-67fa-4c21-9429-4082d20bf48c
NoneType: None
distributed.scheduler - ERROR - Workers don't have p

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-94f60871-7264-42d6-b415-580c18775d95
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-67a2716d-3f4b-471f-aad4-123367f32df4
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-f2b6dfcc-f4af-4693-b213-6f59a416154f
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-c93ac76f-aac1-40ee-b527-500621fcc8c4
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-49e5b03f-1551-4eca-8f19-aa803d20d154
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-2427b5d7-ee89-4cb1-87e2-8ef0f1eb1bdf
NoneType: None
distributed.scheduler - ERROR - Workers don't have p

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-e73a586e-a92f-495f-865c-a8ea41d104f5
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-577a436a-5fb9-4f6b-a042-f6c394809813
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-0efb8d22-21c5-4235-b32b-6b91fae76c70
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-71d85eed-40b8-4aed-bf63-7d92335e6ea1
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-6b7cdf4f-bf77-4c3c-bdf7-6f5a0eabdc34
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-5c9624a7-0f16-4e43-97af-d801fcbf8755
NoneType: None
distributed.scheduler - ERROR - Workers don't have p

...

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-cb06750e-341a-447c-9075-74d355a9ecbb
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-f0c39fac-10c8-41a6-893f-3e769b5b783b
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-b4947a4d-ad8f-4897-8f4c-9b0fffc9e8fc
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-3fc228d8-382a-4c4f-b0f3-4ce7f2de2a6c
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-1ffc8b77-bbd8-4d53-9f29-d35aa11205cc
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-d1e3be84-3913-4ae1-92b7-54d30487d7fb
NoneType: None
distributed.scheduler - ERROR - Workers don't have p

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-c7c1edeb-77a2-497b-829d-ca277c8fc6a0
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-d2254473-9c42-4d5a-8cb4-49c0dfb64f40
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-85afcfa2-3eba-4063-a0f8-86fa39ec6b4d
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-ad5270ed-161d-40a7-8f19-2bb156c05cba
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-91b4cf3b-d952-4ac4-8392-c74e2aeb3b70
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-dfda5d14-6b9d-4b7d-86d5-d36700f00dae
NoneType: None
distributed.scheduler - ERROR - Workers don't have p

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-b07c32f2-0731-41f6-bb67-ee3bf8507fb2
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-275e1953-3ba9-465c-8be2-facd6da15c07
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-0253d835-da35-4960-a8c2-612198bcc723
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-efa9ce96-bd55-41a7-a702-646b339e78e4
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-d295f35a-a671-42f0-8ea9-61ca0191ea23
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-79055d12-55e4-4226-9583-cafb74ef3402
NoneType: None
distributed.scheduler - ERROR - Workers don't have p

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-d7c43986-ae89-4f1d-ab63-0aeadc7fb608
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-d1dba94f-366f-4ed0-acfb-8ade20e4c7f3
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-0964e78e-b0a6-4585-be51-a6ed57be9b45
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-757414c6-1e4e-4762-85b8-434ac74d4535
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-260fbabd-5bcb-4bbe-bf4a-22835106dd1e
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://172.22.179.104:43381'], griddata-90f1e534-8abc-45b9-b93b-d041970243c1
NoneType: None
distributed.scheduler - ERROR - Workers don't have p

CPU times: user 49min 42s, sys: 7min 44s, total: 57min 26s
Wall time: 1h 15min 53s


In [13]:
ds_xr

<xarray.Dataset>
Dimensions:            (bin_size: 1, cross_track: 25, time: 13969, range: 456, vector: 3)
Coordinates:
    alt3d              (range, cross_track, time) float64 dask.array<chunksize=(456, 25, 2000), meta=np.ndarray>
  * cross_track        (cross_track) int64 0 1 2 3 4 5 6 ... 19 20 21 22 23 24
    lat3d              (range, cross_track, time) float64 dask.array<chunksize=(456, 25, 2000), meta=np.ndarray>
    lon3d              (range, cross_track, time) float64 dask.array<chunksize=(456, 25, 2000), meta=np.ndarray>
  * time               (time) datetime64[ns] 2019-09-14T03:22:48.249800 ... 2...
Dimensions without coordinates: bin_size, range, vector
Data variables: (12/58)
    DR                 (bin_size) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    Xat_km             (cross_track, time) float64 dask.array<chunksize=(25, 2000), meta=np.ndarray>
    alt3D              (range, cross_track, time) float64 dask.array<chunksize=(456, 25, 2000), meta=np.ndarray>
    alt_nav            (cross_track, time) float64 dask.array<chunksize=(25, 2000), meta=np.ndarray>
    altsurf14          (cross_track, time) float64 dask.array<chunksize=(25, 2000), meta=np.ndarray>
    altsurf35          (cross_track, time) float64 dask.array<chunksize=(25, 2000), meta=np.ndarray>
    ...                 ...
    vel95s             (range, cross_track, time) float64 dask.array<chunksize=(456, 25, 2000), meta=np.ndarray>
    z95s               (range, cross_track, time) float64 dask.array<chunksize=(456, 25, 2000), meta=np.ndarray>
    zhh14              (range, cross_track, time) float64 dask.array<chunksize=(456, 25, 2000), meta=np.ndarray>
    zhh14SP            (range, cross_track, time) float64 dask.array<chunksize=(456, 25, 2000), meta=np.ndarray>
    zhh35              (range, cross_track, time) float64 dask.array<chunksize=(456, 25, 2000), meta=np.ndarray>
    zhh35SP            (range, cross_track, time) float64 dask.array<chunksize=(456, 25, 2000), meta=np.ndarray>

In [14]:
a = ds_xr.z95s.max(dim=['cross_track', 'range']).compute()

In [15]:
a.max()

<xarray.DataArray 'z95s' ()>
array(64.77657868)